1. EXPLORATIVE DATA ANALYSIS

IMPORTING LIBRARIES

In [ ]:
# Data exploration
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Text processing
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Machine learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix



DATA LOADING

In [ ]:
df = pd.read_csv('edos_labelled_aggregated.csv')

DATA CLEANING

In [ ]:
df = df.drop(columns=["label_category","label_vector"])
df = df.drop_duplicates()
df = df.dropna(subset=['text'])

DATA VISUALIZATION

In [ ]:
df.head(10)

In [ ]:
count = df['label_sexist'].value_counts()
labels = ["sexist", "not sexist"]
sizes = [count.get('sexist', 0), count.get('not sexist', 0)]
colors = ['red', 'blue']

fig, axs = plt.subplots(1, 2, figsize=(18, 6))

axs[0].pie(count,labels=labels, autopct='%1.2f%%')
axs[1].bar(labels, sizes, color=colors)

plt.show()

2. PRE-PROCESSING

In [ ]:
# cleaning with regex

def clean_text(sentence):
    # Convertire il testo in minuscolo
    sentence = sentence.lower()
    # Rimuovere le menzioni (@username)
    sentence = re.sub(r'@\w+', '', sentence)
    # Rimuovere gli URL
    sentence = re.sub(r'http\S+|www\S+|https\S+', '', sentence, flags=re.MULTILINE)
    # Rimuovere hashtag (#hashtag)
    sentence = re.sub(r'#\w+', '', sentence)
    # Rimuovere i numeri
    sentence = re.sub(r'\d+', '', sentence)
    # Rimuovere testo tra parentesi quadre o tonde
    sentence = re.sub(r'\[.*?\]|\(.*?\)', '', sentence)
    # Rimuovere caratteri di nuova linea e tabulazione
    sentence = re.sub(r'\n|\r|\t', ' ', sentence)
    # Rimuovere spazi extra
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    # Rimuovere caratteri speciali
    sentence = re.sub(r'[^\w\s]', '', sentence)
    # Rimuovere tag HTML
    sentence = re.sub(r'<.*?>', '', sentence)
    # Rimuovere indirizzi email
    sentence = re.sub(r'\S+@\S+', '', sentence)
    return sentence

# pipeline preprocessing

def tokenize(text):
    return nltk.word_tokenize(text)

def clean_tokens(tokens):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    return [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

def normalize(tokens):
    return [token.lower() for token in tokens]

def lemmatize(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

def stem(tokens):
    stemmer = nltk.stem.PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

def nlp_pipeline(text):
    tokens = tokenize(text)
    tokens = clean_tokens(tokens)
    tokens = normalize(tokens)
    tokens = lemmatize(tokens)
    tokens = stem(tokens)
    return clean_text(' '.join(tokens))



In [ ]:
# nlp pipeline
df["text"] = df["text"].apply(lambda x: nlp_pipeline(x))

In [ ]:
# print some information about the corpus
print("Size of the corpus:", len(list(df["text"])))
print("Size of the vocabulary:", len(set(list(df["text"]))))
print("Lexical diversity:", len(set(df["text"])) / len(df["text"]))

In [ ]:
df['Nr. Tokens'] = df["text"].apply(lambda x:len(str(x).split()))

In [ ]:
df.head()

3. FEATURE EXTRACTION

In [ ]:
# TF-IDF

# Vettore delle feature del testo
X = list(df["text"])

# # vettore delle labels
y = list(df["label_sexist"])

# Suddivisione del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estrazione delle caratteristiche con TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)



5. MODEL DEFINITION

In [ ]:
# Logistic Regression
model = LogisticRegression()
model.fit(X_train, y_train)


6. EVALUATION AND METRICS

In [ ]:
# evaluate the model using metrics

# Predizioni sul set di test
predictions = model.predict(X_test)

# Valutazione del modello
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, pos_label='sexist')
recall = recall_score(y_test, predictions, pos_label='sexist')
f1 = f1_score(y_test, predictions, pos_label='sexist')
conf_matrix = confusion_matrix(y_test, predictions)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')